In [13]:
import os
from os.path import join
import json
import openai
from IPython.display import display, Markdown

# chatOpenAI within langchain
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI

# prompt template refers to a reproducible way to generate a prompt
from langchain.prompts import ChatPromptTemplate

# LangChain for question answering over a list of documents
from langchain.chains import RetrievalQA

# Load data from a source as Document's. A Document is a piece of text and associated metadata.
from langchain.document_loaders import CSVLoader

# DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory
from langchain.vectorstores import DocArrayInMemorySearch


#takes care of storing embedded data and performing vector search for you
from langchain.indexes import VectorstoreIndexCreator

#pdf loader
from langchain.document_loaders import PyPDFLoader

# QA evaluator
from langchain.evaluation.qa import QAGenerateChain
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import CharacterTextSplitter

from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ConversationBufferMemory

# environment variables

In [2]:
CURRENT_PATH = os.getcwd()
CONFIG_FILE = "config.json"

# Load Config File
with open(join(CURRENT_PATH, CONFIG_FILE)) as file:
    # Load the JSON data
    config = json.load(file)

# Environmental Variables
AZURE_OPENAI_KEY = config['AZURE_OPENAI_KEY']
AZURE_OPENAI_ENDPOINT = config['AZURE_OPENAI_ENDPOINT']
AZURE_ENGINE_NAME = config['AZURE_ENGINE_NAME']
AZURE_ADA_NAME = config['AZURE_ADA_NAME']
AZURE_ADA_ENDPOINT = config['AZURE_ADA_ENDPOINT']
AZURE_ADA_KEY = config['AZURE_ADA_KEY']

# OpenAI ADA API embeddings 
openai_ada_key = AZURE_ADA_KEY
openai_ada_base = AZURE_ADA_ENDPOINT
openai_ada_deployment = AZURE_ADA_NAME
openai_api_type = 'azure'
openai_api_key = AZURE_OPENAI_KEY

In [3]:
ai=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
         ,openai_api_key=AZURE_OPENAI_KEY
         ,openai_api_type='azure'
         ,openai_api_version="2023-05-15"
         ,deployment_name =AZURE_ENGINE_NAME
         ,model = "gpt-3.5-turbo"
         ,temperature=0.1
    )

# load documents

In [4]:
PATH = 'onboarding docs'
FILE = 'chart_valve.json'

In [5]:
# load JSON
import json
chart_file = json.load(open(join(PATH,FILE)))
chart_file=json.dumps(chart_file).split("key")

# embedding & vectorstore

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings(openai_api_key=openai_ada_key
                              , openai_api_base = openai_ada_base
                              , deployment = openai_ada_deployment
                              , openai_api_type = openai_api_type
                              ,chunk_size=1)

In [7]:
from langchain.vectorstores import Chroma

persist_directory='docs/chroma'
vectordb_chart=Chroma.from_texts(
    texts=chart_file,
    embedding=embedding,
    persist_directory=persist_directory
)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Cr

# retriever

In [8]:
retriever = vectordb_chart.as_retriever(search_type="similarity")

# memory

In [9]:
memory_developmentplan = ConversationSummaryMemory(memory_key="chat_history", llm=AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
         ,openai_api_key=AZURE_OPENAI_KEY
         ,openai_api_type='azure'
         ,openai_api_version="2023-05-15"
         ,deployment_name =AZURE_ENGINE_NAME
         ,model = "gpt-3.5-turbo"
         ,temperature=0.1
    ), return_messages=True, input_key="leader")

# plan

In [45]:
prompt_template = """Given the following information, return a list of 5 role specific software/methods/skills for the new employee.

leader: {leader}
Context about leader:{context}
Role:{role}
Context about role:{area}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "leader", "area", "role"]
)
chain_nw = load_qa_chain(ai, chain_type="stuff", prompt=PROMPT, memory=memory_developmentplan, output_key="plan")

In [46]:
#Given the following information, give a recommendation of a development plan for a new employee working under the given leader.
#In addition, return a list of 5 role specific software/methodology/skill for the new employee.

In [47]:
leader="Richard Clark"
role="Business Analyst" 

In [48]:
chain_nw({"input_documents": [retriever.get_relevant_documents(leader)[0]]
          ,"area": retriever.get_relevant_documents(role)
          , "leader": leader, "role":role}, return_only_outputs=True)

{'plan': "1. Financial analysis software: The new employee should be proficient in using financial analysis software to analyze financial data, create reports, and make informed decisions. This skill is important for the role of a Business Analyst as they need to analyze financial data to identify trends and make recommendations for business improvement.\n\n2. Strategic decision-making: The new employee should have strong skills in strategic decision-making. This involves the ability to analyze data, assess risks and opportunities, and make informed decisions that align with the company's goals and objectives. This skill is crucial for a Business Analyst as they need to provide insights and recommendations to support strategic planning and decision-making.\n\n3. Market research: The new employee should be skilled in conducting market research to gather information about industry trends, customer preferences, and competitive landscape. This skill is important for a Business Analyst as t

In [102]:
#memory_developmentplan.chat_memory.messages[-1].content

"Based on the given information, here are 5 role-specific software/methods/skills for the new employee:\n\n1. Financial analysis: The new employee should have expertise in financial analysis to support financial operations, budgeting, and financial planning for the company.\n\n2. Strategic decision-making: The new employee should possess skills in strategic decision-making to assist in making informed decisions that support Valve's growth and investment initiatives.\n\n3. Partnership identification and fostering: The new employee should be skilled in identifying and fostering partnerships with other game developers, publishers, and hardware manufacturers to expand Valve's reach and bring more games and services to the Steam platform.\n\n4. Knowledge of the gaming industry and market trends: The new employee should have extensive knowledge of the gaming industry and market trends to stay updated and make informed business development decisions.\n\n5. Statistical analysis and data visual

# question+answer

In [66]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

convo=ConversationChain(llm=ai, verbose=False, memory=ConversationBufferMemory())

intro="Give me 15 simple, theoretical questions+answers to evaluate knowledge aimed for someone with the role of "
format="; Return the answer in the following format: [{'query': 'insert question here', 'answer': 'insert answer here.'}]"

convo.predict(input=intro+role+format)
questions=convo.memory.chat_memory.messages[-1].content
questions=questions[questions.find('[{'):questions.rfind('}]')+len('}]')]

In [67]:
questions

"[{'query': 'What is the purpose of a SWOT analysis?', 'answer': 'A SWOT analysis is used to assess the strengths, weaknesses, opportunities, and threats of a business or project.'}]\n2. [{'query': 'What is the difference between waterfall and agile methodologies?', 'answer': 'Waterfall is a sequential approach to project management, while agile is an iterative and flexible approach.'}]\n3. [{'query': 'What is the significance of a use case diagram?', 'answer': 'A use case diagram visually represents the interactions between actors and the system, helping to understand system requirements.'}]\n4. [{'query': 'What is the role of a business analyst in requirements gathering?', 'answer': 'A business analyst facilitates the elicitation, documentation, and validation of requirements from stakeholders.'}]\n5. [{'query': 'What is the purpose of a feasibility study?', 'answer': 'A feasibility study assesses the viability of a project, considering technical, economic, and operational factors.'}

In [57]:
s=convo.memory.chat_memory.messages[-1].content
#s[s.find(start)+len(start):s.rfind(end)]

In [53]:
s

"[{'query': 'What are some commonly used financial analysis software?', 'answer': 'Some commonly used financial analysis software include Excel, QuickBooks, and Tableau.'}, {'query': 'How can financial analysis software be used to analyze financial data?', 'answer': 'Financial analysis software can be used to import and organize financial data, perform calculations and ratios, create visualizations and reports, and conduct scenario analysis.'}, {'query': 'How can financial analysis software help in making informed decisions?', 'answer': 'Financial analysis software can help in making informed decisions by providing accurate and up-to-date financial information, allowing for data-driven analysis and forecasting, and facilitating the identification of trends and patterns.'}, {'query': 'What are some key steps involved in strategic decision-making?', 'answer': 'Some key steps involved in strategic decision-making include conducting a SWOT analysis, evaluating potential risks and opportuni

In [28]:
import re
result=re.search('[{(.*)}]', convo.memory.chat_memory.messages[-1].content)

In [30]:
result.group(1)

IndexError: no such group

# google search alternative

In [105]:
#cool mas tem créditos muito limitados; no serpapi experimentei uns minutos e estourei 10 créditos em 100

In [10]:
from langchain.utilities import SerpAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

In [11]:
from langchain.agents import initialize_agent
os.environ["SERPAPI_API_KEY"]='6de7d45e94b2045d72dc0598c65046d475fe817aaeb2afad3e17038ac9154b64'
os.environ["SERPER_API_KEY"] = 'cb0eb32f73a1874b7bd65e2a55c3b6209d14bc07'

In [14]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
os.environ["SERPAPI_API_KEY"]='6de7d45e94b2045d72dc0598c65046d475fe817aaeb2afad3e17038ac9154b64'
os.environ["SERPER_API_KEY"] = 'cb0eb32f73a1874b7bd65e2a55c3b6209d14bc07'
tools = load_tools(["serpapi"], llm=ai) #"serpapi","wikipedia",google-serper
agent_1 = initialize_agent(tools, ai, agent="zero-shot-react-description", verbose=True, memory=ConversationBufferMemory())

In [54]:
planner = load_chat_planner(ai)
executor = load_agent_executor(ai, tools, verbose=False)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=False, memory=ConversationBufferMemory())

In [17]:
agent_1.run("5 technical methods/tools/skills of "+role) 



> Entering new AgentExecutor chain...
I should search for the technical methods, tools, and skills commonly used by business analysts.
Action: Search
Action Input: "technical methods tools skills of Business Analyst"
Observation: Here are some key technical skills that are valuable for business analysts:
Thought:I need to gather more information about these technical skills to provide a comprehensive answer.
Action: Search
Action Input: "technical skills of business analysts"
Observation: Examples of technical skills for business analysts · Proficiency in basic office applications · Knowledge of operating systems · Software testing ...
Thought:I have gathered enough information to provide a comprehensive answer.
Final Answer: Some common technical methods, tools, and skills of a Business Analyst include proficiency in basic office applications, knowledge of operating systems, software testing, data analysis, and database querying.

> Finished chain.


'Some common technical methods, tools, and skills of a Business Analyst include proficiency in basic office applications, knowledge of operating systems, software testing, data analysis, and database querying.'

In [44]:
agent_1.run("set of questions+answers containing 15 technical questions related to "+role) 



> Entering new AgentExecutor chain...
I need to find a set of questions and answers related to Business Analyst.
Action: Search
Action Input: "set of questions and answers for Business Analyst"
Observation: Basic Business Analyst Interview Questions
Thought:This seems like a good resource to start with, but I should keep searching for more comprehensive sets of questions and answers.
Action: Search
Action Input: "comprehensive set of Business Analyst interview questions and answers"
Observation: Top Business Analyst Interview Questions and Answers
Thought:This resource seems to have a more comprehensive set of questions and answers for Business Analyst interviews. I should review these questions to find the ones that are most relevant to my needs.
Action: Search
Action Input: "Business Analyst interview questions and answers pdf"
Observation: 100+ Business Analyst Interview Questions and Answers (2023) · 1) What is a flowchart? · 2) Define the Use Case Model · 3) What does UML stand 

'I have found multiple sets of Business Analyst interview questions and answers that cover various topics and industries. I should review these resources to prepare for my interview.'

In [18]:
agent_1.memory.chat_memory.messages

[HumanMessage(content='5 technical methods/tools/skills of Business Analyst', additional_kwargs={}, example=False),
 AIMessage(content='Some common technical methods, tools, and skills of a Business Analyst include proficiency in basic office applications, knowledge of operating systems, software testing, data analysis, and database querying.', additional_kwargs={}, example=False)]

# option 2 - map_reduce

In [99]:
question_prompt_template = """Given the following information, give a recommendation of a development plan for a new employee working under the given leader.

Leader: {leader}
Context about Leader:{context}
Role:{role}
Context about role:{area}
"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "leader", "area", "role"]
)

combine_prompt_template = """Considering the following development plan, return specific knowledge and concepts that is distinctive from another functions in the company.

Development plan: {summaries}"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries"]
)

In [ ]:
specific examples of concepts and techniques, with detailed explanations, that the new employee should know as a baseline, when starting his role

In [100]:
chain = load_qa_chain(ai, chain_type="map_reduce", question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT, return_map_steps=True)

In [101]:
leader="Richard Clark, Finance Manager"
role="Business Analyst" 

In [98]:
[retriever.get_relevant_documents(leader)[0]]

[Document(page_content='": 15, "department": "Administrative", "title": "Finance Manager", "parent": 10, "name": "Richard Clark", "ask me about": "Richard Clark is the Finance Manager at Valve. He oversees financial operations, budgeting, and financial planning for the company. Richard\'s expertise in financial analysis and strategic decision-making supports Valve\'s growth and investment initiatives."}, {"', metadata={})]

In [103]:
result=chain({"input_documents": [retriever.get_relevant_documents(leader)[0]]
              ,"leader": leader
              ,"area": retriever.get_relevant_documents(role) 
              ,"role":role}, return_only_outputs=True)

In [104]:
result

{'intermediate_steps': ["Based on the given information, a development plan for a new employee working under Richard Clark, the Finance Manager, could include the following steps:\n\n1. Orientation and Familiarization: The new employee should be provided with an orientation session to understand the company's financial operations, budgeting processes, and financial planning strategies. This will help them gain a comprehensive understanding of the finance department's role within the organization.\n\n2. Shadowing and Observation: The new employee should have the opportunity to shadow Richard Clark and observe his day-to-day activities. This will allow them to learn from his expertise in financial analysis and strategic decision-making. They can also observe how he interacts with other departments and stakeholders.\n\n3. Training in Financial Analysis: Since Richard Clark is an expert in financial analysis, it would be beneficial for the new employee to receive training in this area. Thi